# An Introduction to Pandas Part II
### Harsha Kankanamge

In [1]:
import pandas as pd
pd.__version__

'0.23.4'

In [2]:
 # get working (current) directory
import os
os.getcwd()

'C:\\Users\\harsh\\Desktop\\GEG584\\Assignment_3'

In [3]:
os.chdir('data') # change directory to where data file is stored
os.getcwd()

'C:\\Users\\harsh\\Desktop\\GEG584\\Assignment_3\\data'

In [4]:
filename = 'PlacePop_poverty2017.csv'
pop_df = pd.read_csv(filename,encoding='UTF-8')
pop_df.shape # How many rows and columns in the dataframe

(29323, 7)

In [5]:
pop_df.head()

,Id,Id2,Geography,PopEst,PopMoe,Under18Est,Under18Moe
0,1600000US0100100,100100,"Abanda CDP, Alabama",174,158,43,67
1,1600000US0100124,100124,"Abbeville city, Alabama",2412,81,499,118
2,1600000US0100460,100460,"Adamsville city, Alabama",4379,31,950,220
3,1600000US0100484,100484,"Addison town, Alabama",720,198,141,77
4,1600000US0100676,100676,"Akron town, Alabama",318,125,58,38


In [6]:
pop_df.columns

Index(['Id', 'Id2', 'Geography', 'PopEst', 'PopMoe', 'Under18Est',
       'Under18Moe'],
      dtype='object')

In [7]:
fields=[]
for x in [2,5,6]:fields.append(pop_df.columns[x])
fields

['Geography', 'Under18Est', 'Under18Moe']

In [8]:
pop_df = pd.read_csv(filename,usecols=fields)
pop_df.shape

(29323, 3)

### Find places similar in size to Buffalo, NY.
Calculate the z-score comparing the under 18 population of all places to the under 18 population of Buffalo, NY.                
Use this information to retain places with a under 18 population that is not significantly different from Buffalo's.                   

1.Get Buffalo's population (Under18Est and Under18Moe)                                
2.Calculate z-score comparing Buffalo to all other places

In [9]:
bufEst = pop_df.loc[pop_df.Geography=='Buffalo city, New York',['Under18Est']].values[0,0]
bufMoe = pop_df.loc[pop_df.Geography=='Buffalo city, New York',['Under18Moe']].values[0,0]
bufEst,bufMoe

(58617, 1157)

In [10]:
pop_df['zBflo']=(pop_df['Under18Est']-bufEst)/((pop_df['Under18Moe']/1.645)**2+(bufMoe/1.645)**2)**.5
pop_df['zBflo'][:10]

0   -83.140086
1   -82.204616
2   -80.546627
3   -82.956526
4   -83.213173
5   -65.144343
6   -68.131156
7   -75.606464
8   -80.195862
9   -81.682201
Name: zBflo, dtype: float64

In [11]:
pop_df.loc[(pop_df['zBflo'].abs() < 1.645)] # under 18 population same as Buffalo, 90% confidence

,Geography,Under18Est,Under18Moe,zBflo
2405,"Fontana city, California",59811,1396,1.083277
2839,"Moreno Valley city, California",58287,1175,-0.329195
5798,"Urban Honolulu CDP, Hawaii",59530,1069,0.953427
6110,"Aurora city, Illinois",58957,1237,0.330212
15667,"Henderson city, Nevada",59820,1404,1.087743
16959,"Buffalo city, New York",58617,1157,0.000000
24703,"Brownsville city, Texas",57089,1119,-1.561606


## Result
Only seven places have Under 18 population equal to Buffalo's (not surprising).                     

Define medium size places as those with under 18 populations between 30,000 and 80,000.                                   

Find all places with populations >= 30,000 but less than 80,000.                                   

In [12]:
pop_df.loc[pop_df.Geography=='Buffalo city, New York']

,Geography,Under18Est,Under18Moe,zBflo
16959,"Buffalo city, New York",58617,1157,0.0


In [13]:
pop_df['zLB']=(pop_df['Under18Est']-30000)/(pop_df['Under18Moe']/1.645)
pop_df['zUB']=(pop_df['Under18Est']-80000)/(pop_df['Under18Moe']/1.645)
pop_df[:10]

,Geography,Under18Est,Under18Moe,zBflo,zLB,zUB
0,"Abanda CDP, Alabama",43,67,-83.140086,-735.511418,-1963.123358
1,"Abbeville city, Alabama",499,118,-82.204616,-411.263941,-1108.297839
2,"Adamsville city, Alabama",950,220,-80.546627,-217.214773,-591.078409
3,"Addison town, Alabama",141,77,-82.956526,-637.896818,-1706.078636
4,"Akron town, Alabama",58,38,-83.213173,-1296.173421,-3460.647105
5,"Alabaster city, Alabama",8687,501,-65.144343,-69.979810,-234.151467
6,"Albertville city, Alabama",6447,498,-68.131156,-77.800572,-242.961215
7,"Alexander City city, Alabama",3564,310,-75.606464,-140.281355,-405.603935
8,"Alexandria CDP, Alabama",837,257,-80.195862,-186.665895,-506.704805
9,"Aliceville city, Alabama",535,172,-81.682201,-281.801890,-759.999564


In [14]:
lbdf=pop_df.loc[(pop_df.zLB >= -1.645)]
medPlaces = lbdf.loc[(lbdf.zUB < -1.645)].copy() # population within (30000, 80000), 90% confidence
medPlaces.reset_index(inplace=True)
medPlaces = medPlaces[fields]
medPlaces.tail(10)

,Geography,Under18Est,Under18Moe
151,"Arlington CDP, Virginia",39441,213
152,"Chesapeake city, Virginia",57010,202
153,"Newport News city, Virginia",41268,264
154,"Norfolk city, Virginia",48474,185
155,"Richmond city, Virginia",38716,341
156,"Kent city, Washington",30344,1038
157,"Spokane city, Washington",45733,913
158,"Tacoma city, Washington",43458,1119
159,"Vancouver city, Washington",39000,973
160,"Madison city, Wisconsin",40824,787


### Result
This new dataframe contains under 18 population estimates for 161 places similar in size to Buffalo. We are interested in how Buffalo compares to these cities with respect to the percent of the under 16 population below the poverty level.                         

Since we only need the data for these 161 places, join the poverty rate data to this dataframe.

Analyze the results for these places.

In [15]:
filename = 'poverty2017_5yr.csv'
fields = ['Geography', 'Under 18 BelPov Est','Under 18 BelPov MOE' ]
df = pd.read_csv(filename,usecols=fields)
df.columns=["Geography", "est","moe"]
df.columns

Index(['Geography', 'est', 'moe'], dtype='object')

Since the population dataframe and the poverty rate dataframe have a common field (Geography), a simple merge statement suffices.

In [16]:
mergedf = pd.merge(medPlaces,df)
mergedf.tail()

,Geography,Under18Est,Under18Moe,est,moe
156,"Kent city, Washington",30344,1038,21.2,2.8
157,"Spokane city, Washington",45733,913,24.1,2.5
158,"Tacoma city, Washington",43458,1119,21.7,2
159,"Vancouver city, Washington",39000,973,20.5,2.5
160,"Madison city, Wisconsin",40824,787,16.5,2.4


In [17]:
mergedf['est']=mergedf['est'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
mergedf['moe']=mergedf['moe'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [18]:
mergedf.dropna(inplace=True)
mergedf[:5]

,Geography,Under18Est,Under18Moe,est,moe
0,"Birmingham city, Alabama",42202,1287,45.4,2.9
1,"Huntsville city, Alabama",39934,1048,28.7,2.3
2,"Mobile city, Alabama",41863,1086,34.8,2.7
3,"Montgomery city, Alabama",47933,448,33.5,2.4
4,"Anchorage municipality, Alaska",72324,363,11.5,1.5


## Calculate z-score

In [19]:
bufEst = mergedf.loc[mergedf.Geography=='Buffalo city, New York',['est']].values[0,0]
bufMoe = mergedf.loc[mergedf.Geography=='Buffalo city, New York',['moe']].values[0,0]
bufEst,bufMoe

(47.2, 2.4)

In [20]:
mergedf['zBflo2']=(mergedf['est']-bufEst)/((mergedf['moe']/1.645)**2+(bufMoe/1.645)**2)**.5
mergedf['zBflo2'][:10]

0    -0.786599
1    -9.154961
2    -5.646539
3    -6.639880
4   -20.749989
5   -20.131630
6   -25.940385
7    -9.109344
8   -19.829989
9   -16.186344
Name: zBflo2, dtype: float64

In [21]:
mergedf.loc[mergedf.zBflo2 > 1.645,['Geography','est','moe']] # percent poverty greater than Buffalo, 90% confidence

,Geography,est,moe
108,"Rochester city, New York",51.9,2.3


### Result
Only 1 Place in the U.S. similar in size to Buffalo has a higher proportion of population below the poverty level.

How many similarly sized places have a poverty rate that is not significantly different from Buffalo's?

In [22]:
mergedf['zBflo2']=mergedf['zBflo2'].abs()

In [23]:
mergedf.loc[mergedf.zBflo2 <= 1.645,['Geography','est','moe']] # percent poverty similar to Buffalo, 90% confidence

,Geography,est,moe
0,"Birmingham city, Alabama",45.4,2.9
107,"Buffalo city, New York",47.2,2.4
109,"Syracuse city, New York",46.8,2.7
118,"Dayton city, Ohio",49.4,3.2


### Result Only 3!
How many places with under 18 population greater than 80000 have higher poverty rates than Buffalo?

In [24]:
bigdf=pop_df.loc[(pop_df.zUB >= 1.645)].copy() # under 18 population greater than or equal to 80000, 90% confidence
bigdf.shape

(54, 6)

In [25]:
bigMergedf = pd.merge(bigdf,df)
bigMergedf.tail()

,Geography,Under18Est,Under18Moe,zBflo,zLB,zUB,est,moe
49,"Laredo city, Texas",85469,425,35.836409,214.697659,21.168247,40.8,2
50,"San Antonio city, Texas",366098,1669,249.065630,331.264955,281.983948,27.2,0.9
51,"Virginia Beach city, Virginia",100600,244,58.405945,475.971311,138.881148,11.4,1.2
52,"Seattle city, Washington",103576,1479,39.385453,81.834023,26.222123,13.4,1.4
53,"Milwaukee city, Wisconsin",154525,1026,102.023786,199.652656,119.486964,39.8,1.2


In [26]:
bigMergedf['est']=bigMergedf['est'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
bigMergedf['moe']=bigMergedf['moe'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [27]:
bigMergedf['zBflo2']=(bigMergedf['est']-bufEst)/((bigMergedf['moe']/1.645)**2+(bufMoe/1.645)**2)**.5
bigMergedf['zBflo2'][:10]

0   -14.464353
1   -10.717653
2    -8.115333
3   -12.531930
4   -12.137174
5    -4.049138
6   -11.463053
7   -11.876868
8   -11.633789
9   -11.624643
Name: zBflo2, dtype: float64

In [28]:
bigMergedf.loc[bigMergedf.zBflo2 > 1.645,['Geography','est','moe']] # percent poverty greater than Buffalo, 90% confidence

,Geography,est,moe
27,"Detroit city, Michigan",54.5,1.4
35,"Cleveland city, Ohio",51.6,1.8


In [29]:
bigMergedf['zBflo2']=bigMergedf['zBflo2'].abs()
bigMergedf.loc[bigMergedf.zBflo2 <= 1.645,['Geography','est','moe']] # percent poverty similar to Buffalo, 90% confidence

,Geography,est,moe


### Summary
Fifty four large places (Mesa city,Phoenix city, Arizona , Anaheim city, California and more) and 1 similar sized place (Rochester, NY) have under 18 (childrens) poverty rates higher than Buffalo's. Three similary sized places (Birmingham city, Alabama, Syracuse city, New York and Dayton city, Ohio) have poverty rates that are not significantly different from Buffalo's for under 18 (childrens).



# Challenge:

Instead of using total population as the measure of size for the comparisons, use the proportion of children in the population. These values can be calculated from the PlacePop_poverty2017.csv file.

In [30]:
import pandas as pd
df_2=pd.read_csv('PlacePop_poverty2017.csv')

In [31]:
df_2[:5]

,Id,Id2,Geography,PopEst,PopMoe,Under18Est,Under18Moe
0,1600000US0100100,100100,"Abanda CDP, Alabama",174,158,43,67
1,1600000US0100124,100124,"Abbeville city, Alabama",2412,81,499,118
2,1600000US0100460,100460,"Adamsville city, Alabama",4379,31,950,220
3,1600000US0100484,100484,"Addison town, Alabama",720,198,141,77
4,1600000US0100676,100676,"Akron town, Alabama",318,125,58,38


 ### Calculate the estimated proportion of children PctKidEst = (Under18Est/PopEst) and the margin of error

In [32]:
df_2['PctKidEst'] = (df_2['Under18Est']/df_2['PopEst']) 

### PctKidMoe = ((1/PopEst) * sqrt( Under18Moe^2 - PctKidEst^2 * PopMoe^2) 

In [33]:
df_2['PctKidMoe'] = ((1/df_2['PopEst']) * ( df_2['Under18Moe']**2 - (df_2['PctKidEst']**2 * df_2['PopMoe']**2))**0.5)

In [34]:
df_2[:5]

,Id,Id2,Geography,PopEst,PopMoe,Under18Est,Under18Moe,PctKidEst,PctKidMoe
0,1600000US0100100,100100,"Abanda CDP, Alabama",174,158,43,67,0.247126,0.312910
1,1600000US0100124,100124,"Abbeville city, Alabama",2412,81,499,118,0.206882,0.048426
2,1600000US0100460,100460,"Adamsville city, Alabama",4379,31,950,220,0.216945,0.050216
3,1600000US0100484,100484,"Addison town, Alabama",720,198,141,77,0.195833,0.092395
4,1600000US0100676,100676,"Akron town, Alabama",318,125,58,38,0.182390,0.095600


In [35]:
df_2['PctKidEst']=df_2['PctKidEst']*100
df_2['PctKidMoe']=df_2['PctKidMoe']*100

In [36]:
df_2=df_2[["Geography", "PctKidEst","PctKidMoe"]]
df_2[:5]

,Geography,PctKidEst,PctKidMoe
0,"Abanda CDP, Alabama",24.712644,31.291041
1,"Abbeville city, Alabama",20.688226,4.842623
2,"Adamsville city, Alabama",21.694451,5.021630
3,"Addison town, Alabama",19.583333,9.239504
4,"Akron town, Alabama",18.238994,9.560045


### Find places similar in size to Buffalo, NY.
Calculate the z-score comparing the under 18 population of all places to the under 18 population of Buffalo, NY.                
Use this information to retain places with a under 18 population that is not significantly different from Buffalo's.                   

1.Get Buffalo's population (Under18Est and Under18Moe)                                
2.Calculate z-score comparing Buffalo to all other places

In [37]:
bufEst = df_2.loc[df_2.Geography=='Buffalo city, New York',['PctKidEst']].values[0,0]
bufMoe = df_2.loc[df_2.Geography=='Buffalo city, New York',['PctKidMoe']].values[0,0]
bufEst,bufMoe

(23.324022346368714, 0.45658900448776835)

In [38]:
df_2['zBflo2']=(df_2['PctKidEst']-bufEst)/((df_2['PctKidMoe']/1.645)**2+(bufMoe/1.645)**2)**.5
df_2['zBflo2'][:10]

0    0.072993
1   -0.891406
2   -0.531627
3   -0.665180
4   -0.873986
5    3.735851
6    5.010986
7    1.044431
8   -0.964435
9   -0.395239
Name: zBflo2, dtype: float64

In [39]:
df_3=df_2.loc[(df_2['zBflo2'].abs() < 1.645)].copy() # percent under 18 same as Buffalo, 90% confidence

In [40]:
df_3

,Geography,PctKidEst,PctKidMoe,zBflo2
0,"Abanda CDP, Alabama",24.712644,31.291041,0.072993
1,"Abbeville city, Alabama",20.688226,4.842623,-0.891406
2,"Adamsville city, Alabama",21.694451,5.021630,-0.531627
3,"Addison town, Alabama",19.583333,9.239504,-0.665180
4,"Akron town, Alabama",18.238994,9.560045,-0.873986
7,"Alexander City city, Alabama",24.712245,2.138275,1.044431
8,"Alexandria CDP, Alabama",20.379839,5.000982,-0.964435
9,"Aliceville city, Alabama",21.953221,5.687021,-0.395239
10,"Allgood town, Alabama",21.237458,12.134203,-0.282670
11,"Altoona town, Alabama",18.719212,6.598968,-1.145156


## Result
18804 places have percent population under 18 equal to Buffalo'.   
Comapre to poverty rate for under 18 in Buffalo to the rate for these places

In [41]:
filename = 'poverty2017_5yr.csv'
fields = ['Geography', 'Under 18 BelPov Est','Under 18 BelPov MOE' ]
df1 = pd.read_csv(filename,usecols=fields)
df1.columns=["Geography", "est","moe"]
df1.columns

Index(['Geography', 'est', 'moe'], dtype='object')

In [42]:
mergedf1 = pd.merge(df_3,df1)
mergedf1

,Geography,PctKidEst,PctKidMoe,zBflo2,est,moe
0,"Abanda CDP, Alabama",24.712644,31.291041,0.072993,0,43.5
1,"Abbeville city, Alabama",20.688226,4.842623,-0.891406,31.1,15.8
2,"Adamsville city, Alabama",21.694451,5.021630,-0.531627,18.6,14.8
3,"Addison town, Alabama",19.583333,9.239504,-0.665180,51.8,26.4
4,"Akron town, Alabama",18.238994,9.560045,-0.873986,55.2,27.7
5,"Alexander City city, Alabama",24.712245,2.138275,1.044431,45.7,9.5
6,"Alexandria CDP, Alabama",20.379839,5.000982,-0.964435,9.2,11.5
7,"Aliceville city, Alabama",21.953221,5.687021,-0.395239,61.7,18
8,"Allgood town, Alabama",21.237458,12.134203,-0.282670,42.5,32.6
9,"Altoona town, Alabama",18.719212,6.598968,-1.145156,19.1,13.7


In [43]:
mergedf1['est']=mergedf1['est'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
mergedf1['moe']=mergedf1['moe'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [44]:
bufEst = mergedf1.loc[mergedf1.Geography=='Buffalo city, New York',['est']].values[0,0]
bufMoe = mergedf1.loc[mergedf1.Geography=='Buffalo city, New York',['moe']].values[0,0]
bufEst,bufMoe

(47.2, 2.4)

In [45]:
mergedf1['zBflo3']=(mergedf1['est']-bufEst)/((mergedf1['moe']/1.645)**2+(bufMoe/1.645)**2)**.5
mergedf1['zBflo3'][:10]

0   -1.782209
1   -1.657224
2   -3.137862
3    0.285452
4    0.473317
5   -0.251825
6   -5.321012
7    1.313515
8   -0.236522
9   -3.323440
Name: zBflo3, dtype: float64

In [46]:
mergedf1.loc[mergedf1.zBflo3 > 1.645,['Geography','est','moe']] # percent poverty greater than Buffalo, 90% confidence

,Geography,est,moe
43,"Boligee town, Alabama",84.9,17.1
55,"Broomtown CDP, Alabama",100.0,52.0
58,"Camp Hill town, Alabama",86.1,14.5
103,"Dayton town, Alabama",93.8,34.4
156,"Garden City town, Alabama",74.6,25.7
168,"Gordonville town, Alabama",77.8,26.8
189,"Hayneville town, Alabama",75.0,15.7
197,"Hobson City town, Alabama",76.5,15.8
219,"Kellyton town, Alabama",91.2,17.1
244,"Louisville town, Alabama",94.5,8.4


In [47]:
mergedf1['zBflo3']=mergedf1['zBflo3'].abs()
mergedf1.loc[mergedf1.zBflo3 <= 1.645,['Geography','est','moe']] # percent poverty similar to Buffalo, 90% confidence

,Geography,est,moe
3,"Addison town, Alabama",51.8,26.4
4,"Akron town, Alabama",55.2,27.7
5,"Alexander City city, Alabama",45.7,9.5
7,"Aliceville city, Alabama",61.7,18.0
8,"Allgood town, Alabama",42.5,32.6
11,"Anderson town, Alabama",28.9,23.1
13,"Ardmore town, Alabama",29.5,18.0
15,"Ariton town, Alabama",55.6,20.0
17,"Ashland town, Alabama",33.7,20.1
19,"Atmore city, Alabama",33.5,13.5


## Result
6730 places have percent poverty equal to Buffalo where population under 18 similar to Buffalo'.
416 place percent poverty of child greater than Buffalo